Work in progress...

# Itinerant dynamics with the PC-RNN-HC-M model

This notebook provides the necessary source code to simulate itinerant dynamics using a trained PC-RNN-HC-M model. The model is first trained to generate three cyclic patterns using BPTT. We use the Gaussian mixture prior distribution on the hidden causes to randomly switch between the three learned behaviors, we call such dynamics itinerant.

## Model definition

## Dataset

## Model Training

## Itinerancy simulation